In this chapter, we will cover the following topics:

<ul>
    <li>Finding the Dijkstra shortest path with pgRouting</li>
    <li>Finding the Dijkstra shortest path with NetworkX in pure Python</li>
    <li>Generating evacuation polygons based on an indoor shortest path</li>
    <li>Creating centerlines from polygons</li>
    <li>Building an indoor routing system in 3D</li>
    <li>Calculating indoor route walk time</li>
</ul>

## Introduction

Routing has become commonplace on navigation devices for road networks across the world. If you want to know how to drive from point A to point B, simply enter the start address and end address into your navigation software and it will calculate the shortest route for you in seconds.

Here's a scenario you may come across: Route me to Prof. Dr. Smith's office in the Geography Department for my meeting at any university anywhere. Hmm, sorry, there's no routing network available on my navigation software. This is a reminder for you to not to forget to ask for directions on campus for your meeting location.

This chapter is all about routing and, specifically, routing inside large building complexes from office A33, first floor in building E01 to office B55, sixth floor in building P7.

<img src="./50790OS_08_01.jpg" height=400 width=400>

<pre><strong>Note</strong>

BIG IMPORTANT NOTE. Pay attention to the input network dataset used and make sure that it is in the EPSG: 3857 coordinate system, a geometric Cartesian meter system. Routing calculations using world coordinates in EPSG: 4326 must be converted if used by such a system. Also, note that the GeoJSON coordinate system is interpreted by QGIS as EPSG:4326 even though the coordinates are stored in EPSG:3857!</pre>


## 8.1. Finding the Dijkstra shortest path with pgRouting

There are a few Python libraries out there, such as networkX and scikit-image, that can find the shortest path over a raster or NumPy array. We want to focus on routing over a vector source and returning a vector dataset; therefore, pgRouting is a natural choice for us. Custom Python Dijkstra or the A Star (A*) shortest path algorithms exist but one that performs well on large networks is hard to find. The pgRouting extension of PostgreSQL is used by OSM and many other projects and is well tested.

Our example will have us load a Shapefile of an indoor network from one floor for simplicity's sake. An indoor network is comprised of network lines that go along the hallways and open walkable spaces within a building, leading to a door in most cases.
Getting ready

For this recipe, we are going to need to set up our PostGIS database with the pgRouting extension. On a Windows machine, you can install pgRouting by downloading a ZIP file for Postgresql 9.3 at http://winnie.postgis.net/download/windows/pg93/buildbot/. Then, extract the zip file into C:\Program Files\PostgreSQL\9.3\.

For Ubuntu Linux users, the pgRouting website explains the details at http://docs.pgrouting.org/2.0/en/doc/src/installation/index.html#ubuntu-debian.

To enable this extension, you have a couple of options. First off, you can run the command-line psql tool to activate the extension as follows if you have your PostgreSQL running as explained in Chapter 1, Setting Up Your Geospatial Python Environment:

<code>
> psql py_geoan_cb -c "create extension pgrouting"
</code>

You can use the pgAdmin user tool by simply opening up the py_geoan_cb database, right-clicking on Extensions, selecting New Extension..., and in the Name field, scrolling down to find the pgRouting entry and selecting it.

Now we need some data to do our routing calculations. The data used is a Shapefile located in your /ch08/geodata/shp/e01_network_lines_3857.shp folder. Take a look at Chapter 3, Moving Spatial Data from One Format to Another, on how to import the Shapefile or use shp2pgsql. Here is the command-line one-liner using ogr2ogr to import the Shapefile:

<code>
>ogr2ogr -a_srs EPSG:3857 -lco "SCHEMA=geodata" -lco "COLUMN_TYPES=type=varchar,type_id=integer" -nlt MULTILINESTRING -nln ch08_e01_networklines -f PostgreSQL "PG:host=localhost port=5432 user=pluto dbname=py_geoan_cb password=secret" geodata/shp/e01_network_lines_3857.shp
</code>

Note that you either use the same username and password from Chapter 1, Setting Up Your Geospatial Python Environment, or your own defined username and password.

For Windows users, you might need to insert the full path of your Shapefile, something that could look like c:\somepath\geodata\shp\e01_network_lines.shp. We explicitly set the input of the EPSG:3857 Web Mercator because, sometimes, ogr2ogr guesses the wrong projection and in this way, it ensures that it is correct on upload. Another thing to note is that we also explicitly define the output table column types because ogr2ogr uses numeric fields for our integers and this does not go well with pgRouting, so we explicitly pass the comma-separated list of field names and field types.

<pre><strong>Tip</strong>

For a detailed description of how ogr2ogr works, visit http://gdal.org/ogr2ogr.html. </pre>

Our new table includes two fields, one called type and the other, type_id. The type_id variable will store an integer used to identify what kind of network segment we are on, such as stairs, an indoor way, or elevator. The remaining fields are necessary for pgRouting, which is installed as shown in the following code, and include columns called source, target, and cost. The source and target columns both need to be integers, while the cost field is of a double precision type. These types are the requirements of the pgRouting functions.

Let's go ahead and add these fields now to our ch08_e01_networklines table with the help of some SQL queries:

<code>
ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN source INTEGER;
ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN target INTEGER;
ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN cost DOUBLE PRECISION;
ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN length DOUBLE PRECISION;
UPDATE geodata.ch08_e01_networklines set length = ST_Length(wkb_geometry);
</code>

Once the network dataset has its new columns, we need to run the create topology pgr_createTopology()function. This function takes the name of our network dataset, a tolerance value, geometry field name, and a primary key field name. The function will create a new table of points on the LineString intersections, that is, nodes on a network that are in the same schema:

<code>
SELECT public.pgr_createTopology('geodata.ch08_e01_networklines',
        0.0001, 'wkb_geometry', 'ogc_fid');
</code>

The pgr_createTopology function parameters include the name of the networklines LineStrings containing our cost and type fields. The second parameter is the distance tolerance in meters followed by the name of the geometry column and our primary key unique id called ogc_fid.

Now that our tables and environment are set up, this allows us to actually create the shortest path called the Dijkstra route.

To run the Python code, make sure you have the psycopg2 and geojson modules installed as described in Chapter 1, Setting Up Your Geospatial Python Environment.

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2
import json
from geojson import loads, Feature, FeatureCollection

db_host = "localhost"
db_user = "calvin"
db_passwd = "planets"
db_database = "py_test"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, port=db_port,
                        password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

start_x = 1587927.204
start_y = 5879726.142
end_x = 1587947.304
end_y = 5879611.257

# find the start node id within 1 meter of the given coordinate
# used as input in routing query start point
start_node_query = """
    SELECT id FROM geodata.ch08_e01_networklines_vertices_pgr AS p
    WHERE ST_DWithin(the_geom, ST_GeomFromText('POINT(%s %s)',3857), 1);"""

# locate the end node id within 1 meter of the given coordinate
end_node_query = """
    SELECT id FROM geodata.ch08_e01_networklines_vertices_pgr AS p
    WHERE ST_DWithin(the_geom, ST_GeomFromText('POINT(%s %s)',3857), 1);
    """

# get the start node id as an integer
cur.execute(start_node_query, (start_x, start_y))
sn = int(cur.fetchone()[0])

# get the end node id as an integer
cur.execute(end_node_query, (end_x, end_y))
en = int(cur.fetchone()[0])


# pgRouting query to return our list of segments representing
# our shortest path Dijkstra results as GeoJSON
# query returns the shortest path between our start and end nodes above
# using the python .format string syntax to insert a variable in the query
routing_query = '''
    SELECT seq, id1 AS node, id2 AS edge, ST_Length(wkb_geometry) AS cost,
           ST_AsGeoJSON(wkb_geometry) AS geoj
      FROM pgr_dijkstra(
        'SELECT ogc_fid as id, source, target, st_length(wkb_geometry) as cost
         FROM geodata.ch08_e01_networklines',
        {start_node},{end_node}, FALSE, FALSE
      ) AS dij_route
      JOIN  geodata.ch08_e01_networklines AS input_network
      ON dij_route.id2 = input_network.ogc_fid ;
  '''.format(start_node=sn, end_node=en)


# run our shortest path query
cur.execute(routing_query)

# get entire query results to work with
route_segments = cur.fetchall()

# empty list to hold each segment for our GeoJSON output
route_result = []

# loop over each segment in the result route segments
# create the list for our new GeoJSON
for segment in route_segments:
    geojs = segment[4]
    geojs_geom = loads(geojs)
    geojs_feat = Feature(geometry=geojs_geom, properties={'nice': 'route'})
    route_result.append(geojs_feat)

# using the geojson module to create our GeoJSON Feature Collection
geojs_fc = FeatureCollection(route_result)

# define the output folder and GeoJSON file name
output_geojson_route = "../geodata/ch08_shortest_path_pgrouting.geojson"


# save geojson to a file in our geodata folder
def write_geojson():
    with open(output_geojson_route, "w") as geojs_out:
        geojs_out.write(json.dumps(geojs_fc))


# run the write function to actually create the GeoJSON file
write_geojson()

# clean up and close database curson and connection
cur.close()
conn.close()

The resulting query, if you ran it inside pgAdmin, for example, would return the following:

<img src="./50790OS_08_02.jpg" height=400 width=400>

A route needs to be visualized on a map and not as a table. Go ahead and drag and drop your newly created /ch08/geodata/ch08_shortest_path_pgrouting.geojson file into QGIS. If all goes well, you should see this pretty little line, excluding the red arrows and text:

<img src="./50790OS_08_03.jpg" height=400 width=400>

### How it works...

Our code journey starts with setting up our database connection so that we can execute some queries against our uploaded data.

Now we are ready to run some routing, but wait, How do we set the start and end points that we want to route to and from? The natural way to do this is to input and the x, y coordinate pair for the start and end points. Unfortunately, the pgr_dijkstra() function takes only the start and end node IDs. This means that we need to get these node IDs from the new table called ch08_e01_networklines_vertices_pgr. To locate the nodes, we use a simple PostGIS function, ST_Within(), to find the nearest node within one meter from the input coordinate. Inside this query, our input geometry uses the ST_GeomFromText() function so that you can clearly see where things go in our SQL. Now, we'll execute our query and convert the response to an integer value as our node ID. This node ID is then ready for input in the next and final query.

The routing query will return a sequence number, node, edge, cost, and geometry for each segment along our final route. The geometry created is GeoJSON using the ST_AsGeoJSON() PostGIS function that feeds the creation of our final GeoJSON output route.

The pgRouting pgr_dijkstra()function's input arguments include an SQL query, start node ID, end node ID, directed value, and a has_rcost Boolean value. We set the directed and has_rcost values to False, while passing in the start_node and end_node IDs. This query performs a JOIN between the generated route IDs and input network IDs so that we have some geometry output to visualize.

Our journey then ends with processing the results and creating our output GeoJSON file. The routing query has returned a list of individual segments from start to end that aren't in the form of a single LineString, but a set of many LineStrings. This is why we need to create a list and append each route segment to a list by creating our GeoJSON FeatureCollection file.

Here, we use the write_geojson() function to output our final GeoJSON file called ch08_shortest_path_pgrouting.geojson.

<pre><strong>Note</strong>

Note that this GeoJSON file is in the EPSG:3857 coordinate system and is interpreted by QGIS as EPSG:4326, which is incorrect. Geodata for routing, such as OSM data and custom datasets, has lots of possible mistakes, errors, and inconsistencies. Beware that the devil is hiding in the detail of the data this time and not so much in the code.</pre>

Go ahead and drag and drop your GeoJSON file into QGIS to see how your final route looks.

## 8.2. Finding the Dijkstra shortest path with NetworkX in pure Python

This recipe is a pure Python solution to calculate the shortest path on a network. NetworkX is the library we will use with many algorithms to solve the shortest path problem, including Dijkstra (http://networkx.github.io/). NetworkX relies on numpy and scipy to perform some graph calculations and help with performance. In this recipe, we will only use Python libraries to create our shortest path based on the same input Shapefile used in our previous recipe.

### Getting ready

Start with installing NetworkX on your machine with the pip installer as follows:

<code>pip install networkx</code>

For the network graph algorithms, NetworkX requires numpy and scipy, so take a look at Chapter 1, Setting Up Your Geospatial Python Environment, for instructions on these. We also use Shapely to generate our geometry outputs to create GeoJSON files, so check whether you have installed Shapely. One hidden requirement is that GDAL/OGR is used in the back end of NetworkX's import Shapefile function. As mentioned earlier, in Chapter 1, you will find instructions on this subject.

The input data that represents our network is a Shapefile at /ch08/geodata/shp/e01_network_lines_3857.shp, containing our network dataset that is already prepared for routing, so make sure you download this chapter. Now you are ready to run the example.

### How to do it

You need to run this code from the command line to generate the resulting output GeoJSON files that you can open in QGIS, so follow along:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import networkx as nx
import numpy as np
import json
from shapely.geometry import asLineString, asMultiPoint


def get_path(n0, n1):
    """If n0 and n1 are connected nodes in the graph,
    this function will return an array of point
    coordinates along the line linking
    these two nodes."""

    return np.array(json.loads(nx_list_subgraph[n0][n1]['Json'])['coordinates'])


def get_full_path(path):
    """
    Create numpy array line result
    :param path: results of nx.shortest_path function
    :return: coordinate pairs along a path
    """
    p_list = []
    curp = None
    for i in range(len(path)-1):
        p = get_path(path[i], path[i+1])
        if curp is None:
            curp = p
        if np.sum((p[0]-curp)**2) > np.sum((p[-1]-curp)**2):
            p = p[::-1, :]
        p_list.append(p)
        curp = p[-1]
    return np.vstack(p_list)


def write_geojson(outfilename, indata):
    """
    create GeoGJSOn file
    :param outfilename: name of output file
    :param indata: GeoJSON
    :return: a new GeoJSON file
    """

    with open(outfilename, "w") as file_out:
        file_out.write(json.dumps(indata))


if __name__ == '__main__':

    # use Networkx to load a Noded shapefile
    # returns a graph where each node is a coordinate pair
    # and the edge is the line connecting the two nodes

    nx_load_shp = nx.read_shp("../geodata/shp/e01_network_lines_3857.shp")

    # A graph is not always connected, so we take the largest connected subgraph
    # by using the connected_component_subgraphs function.
    nx_list_subgraph = list(nx.connected_component_subgraphs(nx_load_shp.to_undirected()))[0]

    # get all the nodes in the network
    nx_nodes = np.array(nx_list_subgraph.nodes())

    # output the nodes to a GeoJSON file to view in QGIS
    network_nodes = asMultiPoint(nx_nodes)
    write_geojson("../geodata/ch08_final_netx_nodes.geojson",
                  network_nodes.__geo_interface__)

    # this number represents the nodes position
    # in the array to identify the node
    start_node_pos = 30
    end_node_pos = 21

    # Compute the shortest path. Dijkstra's algorithm.
    nx_short_path = nx.shortest_path(nx_list_subgraph,
                                     source=tuple(nx_nodes[start_node_pos]),
                                     target=tuple(nx_nodes[end_node_pos]),
                                     weight='distance')

    # create numpy array of coordinates representing result path
    nx_array_path = get_full_path(nx_short_path)

    # convert numpy array to Shapely Linestring
    out_shortest_path = asLineString(nx_array_path)

    write_geojson("../geodata/ch08_final_netx_sh_path.geojson",
                  out_shortest_path.__geo_interface__)

### How it works...

NetworkX has a nice function called read_shp that inputs a Shapefile directly. However, to start doing this, we need to define the write_geojson function to output our results as GeoJSON files. The input Shapefile is a completely connected network dataset. Sometimes, you may find that your input is not connected and this function call to connected_component_subgraphs finds nodes that are connected, only using these connected nodes. The inner function sets our network to undirected.

<pre><strong>Note</strong>

This function does not create a connected network dataset; this job is left for you to perform in QGIS or some other desktop GIS software. One solution is to execute this in PostgreSQL with the tools provided with the pgRouting extension.</pre>

Now, we'll generate the nodes on our network and export them to GeoJSON. This is, of course, not necessary, but it is nice to see where the nodes are on the map to debug your data. If any problems do occur in generating routes, you can visually identify them quite quickly.

Next up, we set the array position of the start and end node to calculate our route. The NetworkX shortest_path algorithm requires you to define the source and target nodes.

<pre><strong>Tip</strong>

One thing to pay attention to is the fact that the source and target are coordinate pairs within an array of points.</pre>

As nice as this array of points are, we need a path and, hence, the get_path and get_full_path functions are discussed next. Our get_path function takes two input nodes, that is, two pairs of coordinates, and returns a NumPy array of edge coordinates along the line. This is followed closely by the get_full_path function that internally uses the get_path function to output the complete list of all paths and coordinates along all paths.

All the edges and corresponding coordinates are then appended to a new list that needs to be combined—hence, the NumPy vstack function. Inside our for loop, we go through each path, getting the edges and coordinates to build our list that then gets concatenated together as our final NumPy array output.

Shapely was built with NumPy compatibility and, therefore, has an asLineString()function that can directly input a NumPy array of coordinates. Now we have the geometry of our final LineString route and can export it to GeoJSON with our function.

<img src="./50790OS_08_04.jpg" height=300 width=400>

## 8.3. Generating evacuation polygons based on an indoor shortest path

Architects and transportation planners, for example, need to plan where and how many exits a building will require based on various standards and safety policies. After a building is built, a facility manager and security team usually do not have access to this information. Imagine that there is an event to be planned and you want to see what areas can be evacuated within a certain time, which are constrained by your list of exits in the building.

During this exercise, we want to create some polygons for a specific start point inside a major building, showing which areas can be evacuated in 10, 20, 30, and 60 second intervals. We assume that people will walk at 5 km/hr or 1.39 m/s, which is their normal walking speed. If we panic and run, our normal run speed increases to 6.7 m/s or 24.12 km/hr.

Our results are going to generate a set of polygons representing our evacuation zones based on the building hallways. We need to define the start position of where the evacuation begins. This starting point of our calculation is equal to the starting point in our route that was discussed in the previous recipe, Finding the Dijkstra shortest path with NetworkX in pure Python.

<img src="./50790OS_08_05.jpg" height=400 width=400>

This image shows the resulting polygons and points that are generated using our script. The results are styled and visualized using QGIS.

### Getting ready

This example uses the network data loaded by our previous recipe, so make sure that you have loaded this data into your local PostgreSQL database. After you have loaded the data, you will have two tables, geodata.ch08_e01_networklines_vertices_pgr and geodata.ch08_e01_networklines. In combination with these tables, you need a single new Shapefile for our input polygons located at /ch08/geodata/shp/e01_hallways_union_3857.shp, representing the building hallways that are used to clip our resulting distance polygons.

### How to do it...

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2
import shapefile
import json
import shapely.geometry as geometry
from geojson import loads, Feature, FeatureCollection
from shapely.geometry import asShape

# database connection
db_host = "localhost"
db_user = "pluto"
db_passwd = "secret"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, port=db_port,
                        password=db_passwd, database=db_database)
cur = conn.cursor()

def write_geojson(outfilename, indata):
    with open(outfilename, "w") as geojs_out:
        geojs_out.write(json.dumps(indata))

# center point for creating our distance polygons
x_start_coord = 1587926.769
y_start_coord = 5879726.492


# query including two variables for the x, y POINT coordinate
start_node_query = """
    SELECT id
    FROM geodata.ch08_e01_networklines_vertices_pgr AS p
    WHERE ST_DWithin(the_geom,
      ST_GeomFromText('POINT({0} {1})',3857),1);
      """.format(x_start_coord, y_start_coord)

# get the start node id as an integer
# pass the variables
cur.execute(start_node_query)
start_node_id = int(cur.fetchone()[0])

combined_result = []

hallways = shapefile.Reader("../geodata/shp/e01_hallways_union_3857.shp")
e01_hallway_features = hallways.shape()
e01_hallway_shply = asShape(e01_hallway_features)

# time in seconds
evac_times = [10, 20, 30, 60]


def generate_evac_polys(start_node_id, evac_times ):
    """
    
    :param start_node_id: network node id to start from
    :param evac_times: list of times in seconds
    :return: none, generates GeoJSON files
    """

    for evac_time in evac_times:

        distance_poly_query = """
            SELECT seq, id1 AS node, cost, ST_AsGeoJSON(the_geom)
                FROM pgr_drivingDistance(
                        'SELECT ogc_fid AS id, source, target,
                            ST_Length(wkb_geometry)/5000*60*60 AS cost
                         FROM geodata.ch08_e01_networklines',
                        {0}, {1}, false, false
                ) as ev_dist
                JOIN geodata.ch08_e01_networklines_vertices_pgr
                AS networklines
                ON ev_dist.id1 = networklines.id;
            """.format(start_node_id, evac_time)

        cur.execute(distance_poly_query)
        # get entire query results to work with
        distance_nodes = cur.fetchall()

        # empty list to hold each segment for our GeoJSON output
        route_results = []

        # loop over each segment in the result route segments
        # create the list of our new GeoJSON
        for dist_node in distance_nodes:
            sequence = dist_node[0]     # sequence number
            node = dist_node[1]         # node id
            cost = dist_node[2]         # cost value
            geojs = dist_node[3]        # geometry
            geojs_geom = loads(geojs) # create geojson geom
            geojs_feat = Feature(geometry=geojs_geom,
                    properties={'sequence_num': sequence,
                    'node':node, 'evac_time_sec':cost,
                    'evac_code': evac_time})
            # add each point to total including all points
            combined_result.append(geojs_feat)
            # add each point for individual evacuation time
            route_results.append(geojs_geom)

        # geojson module creates GeoJSON Feature Collection
        geojs_fc = FeatureCollection(route_results)

        # create list of points for each evac time
        evac_time_pts = [asShape(route_segment) for route_segment in route_results]

        # create MultiPoint from our list of points for evac time
        point_collection = geometry.MultiPoint(list(evac_time_pts))

        # create our convex hull polyon around evac time points
        convex_hull_polygon = point_collection.convex_hull

        # intersect convex hull with hallways polygon (ch = convex hull)
        cvex_hull_intersect = e01_hallway_shply.intersection(convex_hull_polygon)

        # export convex hull intersection to geojson
        cvex_hull = cvex_hull_intersect.__geo_interface__

        # for each evac time we create a unique GeoJSON polygon
        output_ply = "../geodata/ch08-03_dist_poly_" + str(evac_time) + ".geojson"

        write_geojson(output_ply, cvex_hull)

        output_geojson_route = "../geodata/ch08-03_dist_pts_" + str(evac_time) + ".geojson"

        # save GeoJSON to a file in our geodata folder
        write_geojson(output_geojson_route, geojs_fc )

# create or set of evac GeoJSON polygons based
# on location and list of times in seconds
generate_evac_polys(start_node_id, evac_times)

# final result GeoJSON
final_res = FeatureCollection(combined_result)

# write to disk
write_geojson("../geodata/ch08-03_final_dist_poly.geojson", final_res)

# clean up and close database cursor and connection
cur.close()
conn.close()

### How it works...

The code starts with database boiler plate code plus a function to export the GeoJSON result files. To create an evacuation polygon, we require one input, which is the starting point for the distance calculation polygon on our network. As seen in the previous section, we need to find the node on the network closest to our starting coordinate. Therefore, we run a SQL select to find this node that's within one meter of our coordinate.

Next up, we define the combined_result variable that will hold all the points reachable for all specified evacuation times in our list. Hence, it stores the results of each evacuation time in one single output.

The hallways Shapefile is then prepared as Shapely geometry because we will need it to clip our output polygons to be inside the hallways. We are only interested in seeing which areas can be evacuated within the specified time scales of 10, 20, 30, and 60 seconds. If the area is outside the hallways, you are located outside the building and, well, better said, you are safe.

Now, we will loop through each of our time intervals to create individual evacuation polygons for each time defined in our list. The pgRouting extension includes a function called pgr_drivingDistance(), which returns a list of nodes that are reachable within a specified cost. Parameters for this function include the SQL query that returns id, source, target, and cost columns. Our final four parameters include the start node ID that's represented by the %s variable and equals start_node_id. Then, the evacuation time in seconds stored within the evac_time variable followed by two false values. These last two false values are for the directed route or reverse cost calculation, which we are not using.

<pre><strong>Note</strong>

In our case, the cost is calculated as a time value in seconds based on distance. We assume that you are walking at 5 km/hr. The cost is then calculated as the segment length in meters divided by 5000 m x 60 min x 60 sec to derive a cost value. Then, we pass in the start node ID along with our specified evacuation time in seconds. If you want to calculate in minutes, simply remove one of the x 60 in the equation.</pre>

The geometry of each node is then derived through a SQL JOIN between the vertices table and the result list of nodes with node IDs. Now that we have our set of geometry of points for each node reachable within our evacuation time, it's time to parse this result. Parsing is required to create our GeoJSON output, and it also feeds the points into our combined output, the combined_result variable, and the individual evacuation time polygons that are created with a convex hull algorithm from Shapely.

<pre><strong>Tip</strong>
A better or more realistic polygon could be created using alpha shapes. Alpha shapes form a polygon from a set of points, hugging each point to retain a more realistic polygon that follow the shape of the points. The convex hull simply ensures that all the points are inside the resulting polygon. For a good read on alpha shapes, check out this post by Sean Gillies at http://sgillies.net/blog/1155/the-fading-shape-of-alpha/ and this post at http://blog.thehumangeo.com/2014/05/12/drawing-boundaries-in-python/.

What is included in the code is the alpha shapes module called //ch08/code/alpha_shape.py that you can try out with the input data points created, if you've followed along so far, to create a more accurate polygon.</pre>

Our route_results variable stores the GeoJSON geometry used to create individual convex hull polygons. This variable is then used to populate the list of points for each evacuation set of points. It also provides the source of our GeoJSON export, creating FeatureCollection.

The final calculations include using Shapely to create the convex hull polygon, immediately followed by intersecting this new convex hull polygon with our input Shapefile that represents the building hallways. We are only interested in showing areas to evacuate, which boils down to only areas inside the building, hence the intersection.

The remaining code exports our results to the GeoJSON files in your /ch08/geodata folder. Go ahead and open this folder and drag and drop the GeoJSON files into QGIS to visualize your new results. You will want to grab the following files:

<ul>
    <li>ch08-03_dist_poly_10.geojson</li>
    <li>ch08-03_dist_poly_20.geojson</li>
    <li>ch08-03_dist_poly_30.geojson</li>
    <li>ch08-03_dist_poly_60.geojson</li>
    <li>ch08-03_final_dis_poly.geojson</li>
</ul>

## 8.4. Creating centerlines from polygons

For any routing algorithm to work, we need a set of network LineStrings to perform our shortest path query on. Here, you, of course, have some options, ones that you can download to the OSM data to clean up the roads. Secondly, you could digitize your own set of network lines or, thirdly, you can try to autogenerate these lines.

The generation of this network LineString is of utmost importance and determines the quality and types of routes that we can generate. In an indoor environment, we have no roads and street names; instead, we have hallways, rooms, lounges, elevators, ramps, and stairs. These features are our roads, bridges, and highway metaphors where we want to create routes for people to walk.

How we can create basic network LineStrings from polygons that represent hallways is what we are going to show you in this recipe.

<img src="./50790OS_08_06.jpg" height=400 width=400>

### Getting ready

This exercise requires us to have a plan of some sort in digital form with polygons representing hallways and other open spaces where people could walk. Our hallway polygon is courtesy of the Alpen-Adria-Universität Klagenfurt in Austria. The polygons were simplified to keep the rendering time low. The more complex your input geometry, the longer it will take to process.

We are using the scipy, shapely, and numpy libraries, so read Chapter 1, Setting Up Your Geospatial Python Environment, if you have not done so already. Inside the /ch08/code/ folder, you'll find the centerline.py module containing the Centerline class. This contains the actual code that generates centerlines and is imported by the ch08/code/ch08-04_centerline.py module.

### How to do it...

1. The first task is to create a function to create our centerlines. This is the modified version of the Filip Todic orginal centerlines.py class:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from shapely.geometry import LineString
from shapely.geometry import MultiLineString
from scipy.spatial import Voronoi
import numpy as np


class Centerline(object):
    def __init__(self, inputGEOM, dist=0.5):
        self.inputGEOM = inputGEOM
        self.dist = abs(dist)

    def create_centerline(self):
        """
        Calculates the centerline of a polygon.

        Densifies the border of a polygon which is then represented
        by a Numpy array of points necessary for creating the
        Voronoi diagram. Once the diagram is created, the ridges
        located within the polygon are joined and returned.

        Returns:
            a MultiLinestring located within the polygon.
        """

        minx = int(min(self.inputGEOM.envelope.exterior.xy[0]))
        miny = int(min(self.inputGEOM.envelope.exterior.xy[1]))

        border = np.array(self.densify_border(self.inputGEOM, minx, miny))

        vor = Voronoi(border)
        vertex = vor.vertices

        lst_lines = []
        for j, ridge in enumerate(vor.ridge_vertices):
            if -1 not in ridge:
                line = LineString([
                    (vertex[ridge[0]][0] + minx, vertex[ridge[0]][1] + miny),
                    (vertex[ridge[1]][0] + minx, vertex[ridge[1]][1] + miny)])

                if line.within(self.inputGEOM) and len(line.coords[0]) > 1:
                    lst_lines.append(line)

        return MultiLineString(lst_lines)

    def densify_border(self, polygon, minx, miny):
        """
        Densifies the border of a polygon by a given factor
        (by default: 0.5).

        The function tests the complexity of the polygons
        geometry, i.e. does the polygon have holes or not.
        If the polygon doesn't have any holes, its exterior
        is extracted and densified by a given factor.
        If the polygon has holes, the boundary of each hole 
        as well as its exterior is extracted and densified
        by a given factor.

        Returns:
            a list of points where each point is 
            represented
            by a list of its
            reduced coordinates.

        Example:
            [[X1, Y1], [X2, Y2], ..., [Xn, Yn]
        """

        if len(polygon.interiors) == 0:
            exterior_line = LineString(polygon.exterior)
            points = self.fixed_interpolation(exterior_line, minx, miny)

        else:
            exterior_line = LineString(polygon.exterior)
            points = self.fixed_interpolation(exterior_line, minx, miny)

            for j in range(len(polygon.interiors)):
                interior_line = LineString(polygon.interiors[j])
                points += self.fixed_interpolation(interior_line, minx, miny)

        return points

    def fixed_interpolation(self, line, minx, miny):
        """
        A helping function which is used in densifying
        the border of a polygon.

        It places points on the border at the specified 
        distance. By default the distance is 0.5 (meters)
        which means that the first point will be placed
        0.5 m from the starting point, the second point
        will be placed at the distance of 1.0 m from the
        first point, etc. Naturally, the loop breaks when
        the summarized distance exceeds
        the length of the line.

        Returns:
            a list of points where each point is
            represented by
            a list of its reduced coordinates.

        Example:
            [[X1, Y1], [X2, Y2], ..., [Xn, Yn]
        """

        count = self.dist
        newline = []

        startpoint = [line.xy[0][0] - minx, line.xy[1][0] - miny]
        endpoint = [line.xy[0][-1] - minx, line.xy[1][-1] - miny]
        newline.append(startpoint)

        while count < line.length:
            point = line.interpolate(count)
            newline.append([point.x - minx, point.y - miny])
            count += self.dist

        newline.append(endpoint)

        return newline

2. Now that we have a function that creates centerlines, we need some code to import a Shapefile polygon, run the centerlines script, and export our results to GeoJSON so we that can see it in QGIS:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import json
import shapefile
from shapely.geometry import asShape, mapping
from centerline import Centerline


def write_geojson(outfilename, indata):
    with open(outfilename, "w") as file_out:
        file_out.write(json.dumps(indata))


def create_shapes(shapefile_path):
    '''
    Create our Polygon
    :param shapefile_path: full path to shapefile
    :return: list of Shapely geometries
    '''
    in_ply = shapefile.Reader(shapefile_path)
    ply_shp = in_ply.shapes()

    out_multi_ply = [asShape(feature) for feature in ply_shp]

    print("converting to MultiPolygon: ")

    return out_multi_ply


def generate_centerlines(polygon_shps):
    '''
    Create centerlines
    :param polygon_shps: input polygons
    :return: dictionary of linestrings
    '''
    dct_centerlines = {}

    for i, geom in enumerate(polygon_shps):
        print(" now running Centerline creation")
        center_obj = Centerline(geom, 0.5)
        center_line_shply_line = center_obj.create_centerline()
        dct_centerlines[i] = center_line_shply_line

    return dct_centerlines


def export_center(geojs_file, centerlines):
    '''
    Write output to GeoJSON file
    :param centerlines: input dictionary of linestrings
    :return: write to GeoJSON file
    '''
    with open(geojs_file, 'w') as out:

        for i, key in enumerate(centerlines):
            geom = centerlines[key]
            newline = {'id': key, 'geometry': mapping(geom), 'properties': {'id': key}}

            out.write(json.dumps(newline))


if __name__ == '__main__':

    input_hallways = "../geodata/shp/e01_hallways_small_3857.shp"
    # run our function to create Shapely geometries
    shply_ply_halls = create_shapes(input_hallways)

    # create our centerlines
    res_centerlines = generate_centerlines(shply_ply_halls)
    print("now creating centerlines geojson")

    # define output file name and location
    outgeojs_file = '../geodata/04_centerline_results_final.geojson'

    # write the output GeoJSON file to disk
    export_center(outgeojs_file, res_centerlines)

### How it works...

Starting with centerlines.py that contains the Centerline class, there is a lot going on inside the class. We use the Voronoi polygons and extract ridges as centerlines. To create these Voronoi polygons, we need to convert our polygon into LineStrings representing inner and outer polygon edges. These edges then need to be converted to points to feed the Voronoi algorithm. The points are generated based on a densify algorithm that creates points every 0.5 m along the edge of a polygon and all the way around it. This helps the Voronoi function create a more accurate representation of the polygon, and hence provides a better centerline. On the negative side, the higher this distance is set, the more computing power needed.

The ch08-04_centerline.py code then imports this new Centerline class and actually runs it using our hallways polygon. The input polygons are read from a Shapefile using pyshp. Our generated shapes are then pumped into the generate_centerlines function to output a dictionary of LineStrings representing our centerlines.

That output dictionary is then exported to GeoJSON as we loop over the centerlines and use the standard json.dumps function to export it to our file.

## 8.5. Building an indoor routing system in 3D

How to route through one or multiple buildings or floors is what this recipe is all about. This is, of course, the most complex situation involving complex data collection, preparation, and implementation processes. We cannot go into all the complex data details of collection and transformation from ACAD to PostGIS, for example; instead, the finished data is provided.

To create an indoor routing application, you need an already digitized routing network set of lines representing the areas where people can walk. Our data represents the first and second floor of a university building. The resulting indoor route, shown in the following screenshot, starts from the second floor and travels down the stairs to the first floor, all the way through the building, heading up the stairs again to the second floor, and finally reaching our destination.

<img src="./50790OS_08_07.jpg" width=00 height=400>

### Getting ready

For this recipe, we will need to complete quite a few tasks to prepare for the indoor 3D routing. Here's a quick list of requirements:
<ul>
    <li>A Shapefile for the first floor (/ch08/geodata/shp/e01_network_lines_3857.shp).</li>
    <li>A Shapefile for the second floor (/ch08/geodata/shp/e02_network_lines_3857.shp).</li>
    <li>PostgreSQL DB 9.1 + PostGIS 2.1 and pgRouting 2.0. These were all installed in the Finding the Dijkstra shortest path with pgRouting recipe at the beginning of this chapter.</li>
    <li>Python modules, psycopg2 and geojson.</li>
</ul>

Here is the list of tasks that we need to carry out:

1.     Import the Shapefile of the first floor networklines (skip this if you've completed the earlier recipe that imported this Shapefile) as follows:

<code>ogr2ogr -a_srs EPSG:3857 -lco "SCHEMA=geodata" -lco "COLUMN_TYPES=type=varchar,type_id=integer" -nlt MULTILINESTRING -nln ch08_e01_networklines -f PostgreSQL "PG:host=localhost port=5432 user=postgres dbname=py_geoan_cb password=air" geodata/shp/e01_network_lines_3857.shp</code>

2. Import the Shapefile of the second floor networklines as follows:

<code>ogr2ogr -a_srs EPSG:3857 -lco "SCHEMA=geodata" -lco "COLUMN_TYPES=type=varchar,type_id=integer" -nlt MULTILINESTRING -nln ch08_e02_networklines -f PostgreSQL "PG:host=localhost port=5432 user=postgres dbname=py_geoan_cb password=air" geodata/shp/e02_network_lines_3857.shp</code>

3. Assign routing columns to the first floor networklines (skip this step if you've completed it in the previous recipe):

<code>
    ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN source INTEGER;
    ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN target INTEGER;
    ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN cost DOUBLE PRECISION;
    ALTER TABLE geodata.ch08_e01_networklines ADD COLUMN length DOUBLE PRECISION;
    UPDATE geodata.ch08_e01_networklines set length = ST_Length(wkb_geometry);</code>

4. Assign routing columns to the second floor networklines as follows:

<code>
    ALTER TABLE geodata.ch08_e02_networklines ADD COLUMN source INTEGER;
    ALTER TABLE geodata.ch08_e02_networklines ADD COLUMN target INTEGER;
    ALTER TABLE geodata.ch08_e02_networklines ADD COLUMN cost DOUBLE PRECISION;
    ALTER TABLE geodata.ch08_e02_networklines ADD COLUMN length DOUBLE PRECISION;
    UPDATE geodata.ch08_e02_networklines set length = ST_Length(wkb_geometry);
</code>
    
5. Create pgRouting 3D functions that allow you to route over your 3D networklines. These two PostgreSQL functions are critically important as they reflect the original pgRouting 2D functions that have now been converted to allow 3D routing. The order of installation is also very important, so make sure you install pgr_pointtoid3d.sql first! Both SQL files are located in your /ch08/code/ folder:

<code>psql -U username -d py_geoan_cb -a -f pgr_pointtoid3d.sql</code>

6. Next, install pgr_createTopology3d.sql. This is a modified version of the original that now uses our new pgr_pointtoid3d functions as follows:

<code>psql -U username -d py_geoan_cb -a -f pgr_createTopology3d.sql</code>

7. Now we need to merge our two floor network lines into a single 3D LineString table that we will perform our 3D routing on. This set of SQL commands is stored for you at:

<code>psql -U username -d py_geoan_cb -a -f indrz_create_3d_networklines.sql</code>

The exact creation of the 3D routing table is very important to understand as it allows 3D routing queries. Our code is, therefore, listed as follows with SQL comments describing what we are doing at each step:

<pre><code>
-- if not, go ahead and update
-- make sure tables dont exist

drop table if exists geodata.ch08_e01_networklines_routing;
drop table if exists geodata.ch08_e02_networklines_routing;

-- convert to 3d coordinates with EPSG:3857
SELECT ogc_fid, ST_Force_3d(ST_Transform(ST_Force_2D(st_geometryN(wkb_geometry, 1)),3857)) AS wkb_geometry,
  type_id, cost, length, 0 AS source, 0 AS target
  INTO geodata.ch08_e01_networklines_routing
  FROM geodata.ch08_e01_networklines;

SELECT ogc_fid, ST_Force_3d(ST_Transform(ST_Force_2D(st_geometryN(wkb_geometry, 1)),3857)) AS wkb_geometry,
  type_id, cost, length, 0 AS source, 0 AS target
  INTO geodata.ch08_e02_networklines_routing
  FROM geodata.ch08_e02_networklines;

-- fill the 3rd coordinate according to their floor number
UPDATE geodata.ch08_e01_networklines_routing SET wkb_geometry=ST_Translate(ST_Force_3Dz(wkb_geometry),0,0,1);
UPDATE geodata.ch08_e02_networklines_routing SET wkb_geometry=ST_Translate(ST_Force_3Dz(wkb_geometry),0,0,2);


UPDATE geodata.ch08_e01_networklines_routing SET length =ST_Length(wkb_geometry);
UPDATE geodata.ch08_e02_networklines_routing SET length =ST_Length(wkb_geometry);

-- no cost should be 0 or NULL/empty
UPDATE geodata.ch08_e01_networklines_routing SET cost=1 WHERE cost=0 or cost IS NULL;
UPDATE geodata.ch08_e02_networklines_routing SET cost=1 WHERE cost=0 or cost IS NULL;


-- update unique ids ogc_fid accordingly
UPDATE geodata.ch08_e01_networklines_routing SET ogc_fid=ogc_fid+100000;
UPDATE geodata.ch08_e02_networklines_routing SET ogc_fid=ogc_fid+200000;


-- merge all networkline floors into a single table for routing
DROP TABLE IF EXISTS geodata.networklines_3857;
SELECT * INTO geodata.networklines_3857 FROM
(
(SELECT ogc_fid, wkb_geometry, length, type_id, length*o1.cost as total_cost,
   1 as layer FROM geodata.ch08_e01_networklines_routing o1) UNION
(SELECT ogc_fid, wkb_geometry, length, type_id, length*o2.cost as total_cost,
   2 as layer FROM geodata.ch08_e02_networklines_routing o2))
as foo ORDER BY ogc_fid;

CREATE INDEX wkb_geometry_gist_index
   ON geodata.networklines_3857 USING gist (wkb_geometry);

CREATE INDEX ogc_fid_idx
   ON geodata.networklines_3857 USING btree (ogc_fid ASC NULLS LAST);

CREATE INDEX network_layer_idx
  ON geodata.networklines_3857
  USING hash
  (layer);

-- create populate geometry view with info
SELECT Populate_Geometry_Columns('geodata.networklines_3857'::regclass);

-- update stairs, ramps and elevators to match with the next layer
UPDATE geodata.networklines_3857 SET wkb_geometry=ST_AddPoint(wkb_geometry,
  ST_EndPoint(ST_Translate(wkb_geometry,0,0,1)))
  WHERE type_id=3 OR type_id=5 OR type_id=7;
-- remove the second last point
UPDATE geodata.networklines_3857 SET wkb_geometry=ST_RemovePoint(wkb_geometry,ST_NPoints(wkb_geometry) - 2)
  WHERE type_id=3 OR type_id=5 OR type_id=7;


-- add columns source and target
ALTER TABLE geodata.networklines_3857 add column source integer;
ALTER TABLE geodata.networklines_3857 add column target integer;
ALTER TABLE geodata.networklines_3857 OWNER TO postgres;

-- we dont need the temporary tables any more, delete them
DROP TABLE IF EXISTS geodata.ch08_e01_networklines_routing;
DROP TABLE IF EXISTS geodata.ch08_e02_networklines_routing;

-- remove route nodes vertices table if exists
DROP TABLE IF EXISTS geodata.networklines_3857_vertices_pgr;
-- building routing network vertices (fills source and target columns in those new tables)
SELECT public.pgr_createTopology3d('geodata.networklines_3857', 0.0001, 'wkb_geometry', 'ogc_fid');
</code></pre>

Wow, that was a lot of stuff to get through, and now we are actually ready to run and create some 3D routes. Hurray!

### How to do it...

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2
import json
from geojson import loads, Feature, FeatureCollection

db_host = "localhost"
db_user = "pluto"
db_passwd = "secret"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, port=db_port,
                        password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# define our start and end coordinates in EPSG:3857
# set start and end floor level as integer 0,1,2 for example
x_start_coord = 1587848.414
y_start_coord = 5879564.080
start_floor = 2

x_end_coord = 1588005.547
y_end_coord = 5879736.039
end_floor = 2


# find the start node id within 1 meter of the given coordinate
# select from correct floor level using 3D Z value
# our Z Value is the same as the floor number as an integer
# used as input in routing query start point
start_node_query = """
    SELECT id FROM geodata.networklines_3857_vertices_pgr AS p
    WHERE ST_DWithin(the_geom, ST_GeomFromText('POINT(%s %s)',3857), 1)
    AND ST_Z(the_geom) = %s;"""

# locate the end node id within 1 meter of the given coordinate
end_node_query = """
    SELECT id FROM geodata.networklines_3857_vertices_pgr AS p
    WHERE ST_DWithin(the_geom, ST_GeomFromText('POINT(%s %s)',3857), 1)
    AND ST_Z(the_geom) = %s;"""

# run our query and pass in the 3 variables to the query
# make sure the order of variables is the same as the 
# order in your query
cur.execute(start_node_query, (x_start_coord, y_start_coord, start_floor))
start_node_id = int(cur.fetchone()[0])

# get the end node id as an integer
cur.execute(end_node_query, (x_end_coord, y_end_coord, end_floor))
end_node_id = int(cur.fetchone()[0])


# pgRouting query to return our list of segments representing
# our shortest path Dijkstra results as GeoJSON
# query returns the shortest path between our start and end nodes above
# in 3D traversing floor levels and passing in the layer value = floor

routing_query = '''
    SELECT seq, id1 AS node, id2 AS edge, ST_Length(wkb_geometry) AS cost, layer,
           ST_AsGeoJSON(wkb_geometry) AS geoj
      FROM pgr_dijkstra(
        'SELECT ogc_fid as id, source, target, st_length(wkb_geometry) AS cost, layer
         FROM geodata.networklines_3857',
        %s, %s, FALSE, FALSE
      ) AS dij_route
      JOIN  geodata.networklines_3857 AS input_network
      ON dij_route.id2 = input_network.ogc_fid ;
  '''


# run our shortest path query
cur.execute(routing_query, (start_node_id, end_node_id))

# get entire query results to work with
route_segments = cur.fetchall()

# empty list to hold each segment for our GeoJSON output
route_result = []

# loop over each segment in the result route segments
# create the list of our new GeoJSON
for segment in route_segments:
    print segment
    seg_cost = segment[3]     # cost value
    layer_level = segment[4]  # floor number
    geojs = segment[5]        # geojson coordinates
    geojs_geom = loads(geojs) # load string to geom
    geojs_feat = Feature(geometry=geojs_geom, properties={'floor': layer_level, 'cost': seg_cost})
    route_result.append(geojs_feat)

# using the geojson module to create our GeoJSON Feature Collection
geojs_fc = FeatureCollection(route_result)

# define the output folder and GeoJSON file name
output_geojson_route = "../geodata/ch08_indoor_3d_route.geojson"


# save geojson to a file in our geodata folder
def write_geojson():
    with open(output_geojson_route, "w") as geojs_out:
        geojs_out.write(json.dumps(geojs_fc))

# run the write function to actually create the GeoJSON file
write_geojson()

# clean up and close database curson and connection
cur.close()
conn.close()

### How it works...

Using the psycopg2 module, we can connect to our fancy new tables in the database and run some queries. The first query set finds the start and end nodes based on the x, y, and Z elevation values. The Z value is VERY important; otherwise, the wrong node will be selected. The Z value corresponds one to one with a layer/floor value. The 3D elevation data assigned to our networklines_3857 dataset is simply one meter for floor one and two meters for floor two. This keeps things simple and easy to remember without actually using the real height of the floors, which, of course, you could do if you want to.

Our 3D routing is then able to run like any other normal 2D routing query because the data is now in 3D, thanks to our two new pgRouting functions. The query goes through, selects our data, and returns a nice GeoJSON string.

You have seen the remaining code before. It exports the results to a GeoJSON file on disk so that you can open it in QGIS for viewing. We've managed to add a couple of properties to the new GeoJSON file, including the floor number, cost in terms of distance, and the route segment type that identifies whether a segment is an indoor way or is in the form of stairs.

## 8.6. Calculating indoor route walk time

Our indoor routing application would not be complete without letting us know how long it would take to walk to our indoor walk now, would it? We will create a couple of small functions that you can insert into your code in the previous recipe to print out the route walk times.

### How to do it...

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

def format_walk_time(walk_time):
    """
    takes argument: float walkTime in seconds
    returns argument: string time  "xx minutes xx seconds"
    """
    if walk_time > 0.0:
        return str(int(walk_time / 60.0)) + " minutes " + str(int(round(walk_time % 60))) + " seconds"
    else:
        return "Walk time is less than zero! Something is wrong"


def calc_distance_walktime(rows):
    """
    calculates distance and walk_time.
    rows must be an array of linestrings --> a route, retrieved from the DB.
    rows[5]: type of line (stairs, elevator, etc)
    rows[3]: cost as length of segment
    returns a dict with key/value pairs route_length, walk_time
    """

    route_length = 0
    walk_time = 0

    for row in rows:

        route_length += row[3]
        #calculate walk time
        if row[5] == 3 or row[5] == 4:  # stairs
            walk_speed = 1.2 # meters per second m/s
        elif row[5] == 5 or row[5] == 6:  # elevator
            walk_speed = 1.1  # m/s
        else:
            walk_speed = 1.39 # m/s

        walk_time += (row[3] / walk_speed)

    length_format = "%.2f" % route_length
    real_time = format_walk_time(walk_time)
    print {"route_length": length_format, "walk_time": real_time}

Your results should show you a dictionary as follows:

<code>
{'walk_time': '4 minutes 49 seconds', 'route_length': '397.19'}
</code>
    
Here, it is assumed that you have placed these functions into our previous recipe and have called the function to print the results to the console.

### How it works...

We have two simple functions to create walk times for our indoor routes. The first function, called format_walk_time(), simply takes the resulting time and converts it to a human-friendly form, showing the minutes and seconds, respectively, that are required for output.

The second function, calc_distance_walktime(), does the work, expecting a list object including the distance. This distance then gets summed for each route segment into a total distance value that's stored in the route_length variable. Our real_time variable is then created by calling upon the format_walk_time function that passes in the walk_time value in seconds.

Now you have a sophisticated indoor route with specified walk times for your application.